# Import Liberaries

In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install virtualenv

In [ ]:
# !virtualenv /content/drive/MyDrive/colab_env

In [2]:
# !source /content/drive/MyDrive/colab_env/bin/activate; pip install netCDF4 geopandas pyet rasterio wxee geemap rioxarray rasterio virtualenv
!source /content/drive/MyDrive/colab_env/bin/activate

In [3]:
import os
import sys
sys.path.append("/content/drive/MyDrive/colab_env/lib/python3.10/site-packages")
import pandas as pd
import numpy as np
import netCDF4 as nc
import xarray as xr
import geopandas as gpd
import pyet
import pickle
import pyproj
import ee
import wxee
import geemap
import itertools
import rasterio
from osgeo import gdal
import rioxarray



sys.path.append('/content/drive/MyDrive/WaterBalance_new')

from qdwb.evapotranspiration.et import *
from qdwb.evapotranspiration.asset import *
from qdwb.evapotranspiration.convert import *
from qdwb.coordinate.extract import *
from qdwb.primary_surface_flow.primary_surface_flow import *
from qdwb.primary_surface_flow.asset import *
from qdwb.soil_content.soil import *

## Authorize in Google Earth Engine

In [4]:
service_account = 'test-175@ee-mohammadnejadmehdi77.iam.gserviceaccount.com'

credentials = ee.ServiceAccountCredentials(
    email=service_account,
    key_file='/content/drive/MyDrive/private-key.json'
)

ee.Initialize(credentials)

# Variables

In [5]:
PATH_DATA = "/content/drive/MyDrive/WaterBalance_new/assets"

# mashhad
LAT_MIN = 35.80
LAT_MAX = 37.11
LON_MIN = 58.31
LON_MAX = 60.14

# USA
# LAT_MIN = 43.30
# LAT_MAX = 44.60
# LON_MIN = -109.40
# LON_MAX = -107.40

# Functions

In [6]:
def mask_nc_file(
    nc,
    variable,
    lat_min,
    lat_max,
    lon_min,
    lon_max
):
    with xr.open_dataset(nc) as xr_nc:
        result = xr_nc.sel(
            lat = slice(lat_max, lat_min),
            lon = slice(lon_min, lon_max),
        )
    return result

# Load Data

In [8]:
# degree_Celsius - Max air temperature
tmax = mask_nc_file(
    nc = PATH_DATA + "/nc/Tmax_2022031.nc",
    variable = "air_temperature",
    lat_min = LAT_MIN,
    lat_max = LAT_MAX,
    lon_min = LON_MIN,
    lon_max = LON_MAX
)
# degree_Celsius - Min air temperature
tmin = mask_nc_file(
    nc = PATH_DATA + "/nc/Tmin_2022031.nc",
    variable = "air_temperature",
    lat_min = LAT_MIN,
    lat_max = LAT_MAX,
    lon_min = LON_MIN,
    lon_max = LON_MAX
)
# degree_Celsius - Mean air temperature
tmean = mask_nc_file(
    nc = PATH_DATA + "/nc/Temp_2022031.nc",
    variable = "air_temperature",
    lat_min = LAT_MIN,
    lat_max = LAT_MAX,
    lon_min = LON_MIN,
    lon_max = LON_MAX
)
# m s-1 - wind_speed
wind = mask_nc_file(
    nc = PATH_DATA + "/nc/wind_2022031.nc",
    variable = "wind_speed",
    lat_min = LAT_MIN,
    lat_max = LAT_MAX,
    lon_min = LON_MIN,
    lon_max = LON_MAX
)
# W m-2 - downward_shortwave_radiation
Rns = mask_nc_file(
    nc = PATH_DATA + "/nc/SWd_2022031.nc",
    variable = "downward_shortwave_radiation",
    lat_min = LAT_MIN,
    lat_max = LAT_MAX,
    lon_min = LON_MIN,
    lon_max = LON_MAX
)
# kg kg-1 - specific_humidity
specific_humidity = mask_nc_file(
    nc = PATH_DATA + "/nc/SpecHum_2022031.nc",
    variable = "specific_humidity",
    lat_min = LAT_MIN,
    lat_max = LAT_MAX,
    lon_min = LON_MIN,
    lon_max = LON_MAX
)
# Pa - surface_pressure
surface_pressure = mask_nc_file(
    nc = PATH_DATA + "/nc/Pres_2022031.nc",
    variable = "surface_pressure",
    lat_min = LAT_MIN,
    lat_max = LAT_MAX,
    lon_min = LON_MIN,
    lon_max = LON_MAX
)
# mm d-1 - precipitation
P = mask_nc_file(
    nc = PATH_DATA + "/nc/P_2022031.nc",
    variable = "precipitation",
    lat_min = LAT_MIN,
    lat_max = LAT_MAX,
    lon_min = LON_MIN,
    lon_max = LON_MAX
)
# W m-2 - downward_longwave_radiation
Rnl = mask_nc_file(
    nc = PATH_DATA + "/nc/LWd_2022031.nc",
    variable = "downward_longwave_radiation",
    lat_min = LAT_MIN,
    lat_max = LAT_MAX,
    lon_min = LON_MIN,
    lon_max = LON_MAX
)

usa = gpd.read_file(PATH_DATA + "/shape/usa.shp")
mashhad = gpd.read_file(PATH_DATA + "/shape/mahdoode_mashhad.shp")

# initial soil moisture

In [ ]:
crs = 4326
shape = ee.Geometry.Rectangle([LON_MAX,LAT_MIN,LON_MIN,LAT_MAX])


soil_era_evaporation = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR").filter(ee.Filter.date('2022-01-31', '2022-02-01')).select('volumetric_soil_water_layer_1').mean().clip(shape)

soil_era_transpiration = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR").filter(ee.Filter.date('2022-01-31', '2022-02-01')).select('volumetric_soil_water_layer_3').mean().clip(shape)

soil_era_transition = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR").filter(ee.Filter.date('2022-01-31', '2022-02-01')).select('volumetric_soil_water_layer_4').mean().clip(shape)


soil_era_evaporation = soil_era_evaporation.set("system:time_start", ee.Date("2022-01-31"))
soil_era_transpiration = soil_era_transpiration.set("system:time_start", ee.Date("2022-01-31"))
soil_era_transition = soil_era_transition.set("system:time_start", ee.Date("2022-01-31"))
